In [3]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1"
)

completion = client.chat.completions.create(
    model="mistral-7b/",
    messages=[
        {"role": "user", "content": "Tell me a short story about king and queen"}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content=" Title: The King's Compassionate Queen\n\nOnce upon a time, in the kingdom of Serendil, King Elden was a mighty, fair, and just ruler. He was known throughout the land for his wisdom and kindness. By his side, Queen Eirlys, a woman of great beauty and grace, not only was the wife of the king, but also his trusted advisor and the heart of the kingdom.\n\nOne day, a plague spread across the kingdom, affecting both the royal family and common folk alike. The king fell mortally ill, leaving the kingdom in turmoil as Queen Eirlys took charge to ensure the well-being of her people.\n\nQueen Eirlys' boundless compassion shone like a beacon, inspiring hope in the hearts of her people. She worked tirelessly, taking care of the sick, providing food to the hungry, and organizing doctors and healers to treat the afflicted. Despite her own grief, her strength never wavered.\n\nDays turned into weeks, and Queen Eirlys' persistence paid off. Slowly but surely, the plagu

In [2]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1"
)

completion = client.chat.completions.create(
    model="olmOCR-7B-0225-preview",
    messages=[
        {"role": "user", "content": "Answer with just city name. What i capital of india? "}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Delhi', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], reasoning_content=None)


In [ ]:
import json
import random
from openai import OpenAI

# Load ShareGPT data
with open("//home/skamalj/dev/vllm/ShareGPT_V3_unfiltered_cleaned_split.json", "r") as f:
    data = json.load(f)

# Flatten and filter valid user-only prompts
def extract_user_prompts(convo):
    return [msg["value"] for msg in convo["conversations"] if msg["from"] == "human"]

# Sample 100 random prompts
random_samples = random.sample(data, 25)

# Prepare OpenAI client (local server)
client = OpenAI(base_url="http://localhost:8000/v1")

# Iterate over samples
for idx, sample in enumerate(random_samples):
    prompts = extract_user_prompts(sample)
    if not prompts:
        continue

    # Take only the **first** user prompt per sample
    first_prompt = prompts[0].strip()
    print(f"Processing sample #{idx + 1} with prompt: {first_prompt}")

    try:
        completion = client.chat.completions.create(
            model="my-mistral-model",
            messages=[{"role": "user", "content": first_prompt}]
        )

        print(f"Sample #{idx + 1}")
        print("Prompt:", first_prompt)
        print("Response:", completion.choices[0].message.content.strip())
        print("-" * 50)

    except Exception as e:
        print(f"Error on sample #{idx + 1}: {e}")


> Download  data file from [here](https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered)

In [ ]:
import json
import random
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load ShareGPT data
with open("/home/skamalj/dev/hfdata/ShareGPT_V3_unfiltered_cleaned_split.json", "r") as f:
    data = json.load(f)

# Flatten and filter valid user-only prompts
def extract_user_prompts(convo):
    return [msg["value"] for msg in convo["conversations"] if msg["from"] == "human"]

# Sample 100 random prompts
random_samples = random.sample(data, 500)
client = OpenAI(base_url="http://localhost:8000/v1")

# Prepare request payloads
requests = []
for sample in random_samples:
    prompts = extract_user_prompts(sample)
    if prompts:
        requests.append(prompts[0].strip())

# Function to run a single request
def run_request(idx, prompt):
    try:
        completion = client.chat.completions.create(
            model="my-mistral-model",
            messages=[{"role": "user", "content": prompt}]
        )
        return idx, prompt, completion.choices[0].message.content.strip()
    except Exception as e:
        return idx, prompt, f"ERROR: {e}"

# Run in parallel to trigger batching in vLLM
with ThreadPoolExecutor(max_workers=32) as executor:
    futures = [executor.submit(run_request, idx, prompt) for idx, prompt in enumerate(requests)]
    for future in as_completed(futures):
        idx, prompt, response = future.result()
        print(f"Sample #{idx + 1}")
        print("Prompt:", prompt)
        print("Response:", response)
        print("-" * 50)
